# Initial investigation of MNIST data using openCV
This notebook contains the initial exploration of the MNIST-dataset using open CV


## Load data
The first step is simply to load the data and extract parameters surrounding the data

In [2]:
%matplotlib
import cv2 as cv
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.datasets.mnist as mnist
import numpy as np
import numpy.random as rnd

Using matplotlib backend: Qt5Agg


In [5]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
num_train = x_train.shape[0]
num_test = x_test.shape[0]
img_width = x_train.shape[1]
img_height = x_train.shape[2]

## Inspect data
Just take a random image and display it


In [6]:
N = 4
rnd_i = rnd.permutation(num_train)[0:N]
fig, axs = plt.subplots(1, N)
for i in range(N):
    axs[i].imshow(x_train[rnd_i[i]])
    axs[i].set_title("Number is " + str(y_train[rnd_i[i]]))


## Preprocess the data



In [7]:
digit = 5
X = keras.utils.normalize(x_train[y_train == digit,:], axis=1)
Y = y_train[y_train == digit]

X = keras.utils.normalize(x_train, axis=1)
Y = y_train

## Set up models

In [8]:


""" Setup encoder"""
encoder_input = keras.layers.Input((img_width, img_height))
net = keras.layers.Reshape((img_width, img_height,1))(encoder_input)
net = keras.layers.Conv2D(8, (3, 3), activation='relu')(net)
net = keras.layers.MaxPooling2D((2, 2), padding='same')(net)
net = keras.layers.Conv2D(16, (3, 3), activation='relu')(net)
net = keras.layers.MaxPooling2D((2, 2), padding='same')(net)
net = keras.layers.Conv2D(32, (3, 3), activation='relu')(net)
net = keras.layers.MaxPooling2D((2, 2), padding='same')(net)
encoder_output = keras.layers.Flatten()(net)
encoder = keras.Model(encoder_input, encoder_output)
encoder.summary()

""" Setup decoder"""
decoder_input =keras.layers.Input(encoder.output.shape[1:])
net = keras.layers.Dense(512,activation = tf.nn.relu)(decoder_input)
net = keras.layers.Dense(512,activation = tf.nn.relu)(net)
net = keras.layers.Dense(img_width*img_height, activation = tf.nn.sigmoid)(net)

net = keras.layers.Reshape((2,2,32))(decoder_input)
net = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(net)
net = keras.layers.UpSampling2D((2, 2))(net)
net = keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(net)
net = keras.layers.UpSampling2D((2, 2))(net)
net = keras.layers.Conv2D(4, (3, 3), activation='relu', padding='same')(net)
net = keras.layers.UpSampling2D((2, 2))(net)
net = keras.layers.Conv2D(2, (3, 3), activation='relu')(net)
net = keras.layers.UpSampling2D((2, 2))(net)
decoder_output = keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(net)
#decoder_output = keras.layers.Reshape((img_width, img_height), input_shape=(img_width*img_height,))(net)

decoder = keras.Model(decoder_input, decoder_output)
decoder.summary()
""" Setup autocoder"""

image_input = keras.layers.Input((img_width, img_height))
encoded = encoder(image_input)
decoded = decoder(encoded)
autocoder = keras.Model(image_input, decoded)


autocoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')
autocoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 32)          4640  

In [9]:
autocoder.fit(X,X, epochs = 2)

Epoch 1/2
1875/1875 [==============================] - 59s 30ms/step - loss: 0.5417
Epoch 2/2
1875/1875 [==============================] - 63s 33ms/step - loss: 0.2622


## Evaluate model

In [7]:
X_hat = autocoder.predict(X)


In [12]:
M = encoder.layers[2].get_weights()
N = decoder.layers[1].get_weights()

print(M[0].shape)
print(M[1].shape)


fig,axs = plt.subplots(2,2)
axs[0,0].imshow(M[0])
axs[0,1].imshow(np.tile(M[1],(200,1)))
axs[1,0].imshow(N[0])
axs[1,1].imshow(np.tile(N[1],(200,1)))

(3, 3, 1, 8)
(8,)


TypeError: Invalid shape (3, 3, 1, 8) for image data

In [74]:
b = N[0]@M[1]+N[1]
fig,axs = plt.subplots(1,4)

gr0 = axs[0].imshow(np.reshape(M[1],(28,28)))
plt.colorbar(gr0, ax = axs[0])
gr1 = axs[1].imshow(np.reshape(N[1],(28,28)))
plt.colorbar(gr1, ax = axs[1])
gr2 = axs[2].imshow(np.reshape(b,(28,28)))
plt.colorbar(gr2, ax = axs[2])
gr3 = axs[3].imshow(N[0]@M[0])
plt.colorbar(gr3, ax = axs[3])


In [9]:
N = 4
rnd_i = rnd.permutation(X.shape[0])[0:N]
fig, axs = plt.subplots(2, N)
for i in range(N):
    axs[0,i].imshow(X_hat[rnd_i[i]],vmin = 0, vmax = 1.0)
    axs[0,i].set_title("Number is " + str(Y[rnd_i[i]]))
    axs[1,i].imshow(X[rnd_i[i]],vmin = 0, vmax = 1.0)
    axs[1,i].set_title("Number is " + str(Y[rnd_i[i]]))
    


## Setup latentspace


In [118]:
class LatentDigit:
    def __init__(self, model, fig,
                              latent_boundary = (0.0, 1.0, 0.0, 1.0)):
        self.point = np.array([0.1,  0.5])
        self.model = model
        self.image = self.model.predict(np.array([self.point]))[0]
        
        self.fig = fig
        self.fig.subplots(1,2)
        self.latent_boundary = latent_boundary
        self.setup_latent_space()
        
        self.click_tol = 0.05
        self.draw()
        self.fig.canvas.mpl_connect('button_press_event', self.button_press_callback)
        self.fig.canvas.mpl_connect('motion_notify_event',self.motion_notify_callback)
        self.fig.canvas.mpl_connect('button_release_event', self.button_release_callback)
        self.fig.canvas.mpl_connect('key_press_event', self.key_press_callback)

        self.point_selected = False        
    
    def setup_latent_space(self):
        self.fig.axes[0].set_xlim(self.latent_boundary[0], self.latent_boundary[1])
        self.fig.axes[0].set_ylim(self.latent_boundary[2], self.latent_boundary[3])       
        
    
    def draw(self):
        self.fig.axes[0].cla()
        self.setup_latent_space()
        self.fig.axes[0].scatter(self.point[0], self.point[1])
        self.fig.axes[1].imshow(self.image,vmin = 0, vmax = 1.0)
        self.fig.canvas.draw_idle()
    
    def button_press_callback(self, event):
        self.update_point_select(event)
    
    def update_point_select(self, event):
        click = np.array([event.xdata, event.ydata])
        dist = np.linalg.norm(click-self.point)
        self.point_selected =  dist < self.click_tol
    
    def motion_notify_callback(self, event):
        if not self.point_selected: return
        self.update_point(event)
        self.update_digit()
        self.draw()
    
    def button_release_callback(self, event):
        self.point_selected = False
    
    def update_point(self, event):
        self.point = np.array([event.xdata, event.ydata])

    def update_digit(self):
        self.image = self.model.predict(np.array([self.point]))[0]
    
    def key_press_callback(self, event):
        if not event.key == " ": return
        self.update_point(event)
        self.update_digit()
        self.draw()


In [119]:

ld = LatentDigit(decoder, plt.figure())
plt.show()

Traceback (most recent call last):
  File "C:\Users\joelw\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-118-c0a6d211a3ff>", line 60, in key_press_callback
    self.update_digit()
  File "<ipython-input-118-c0a6d211a3ff>", line 55, in update_digit
    self.image = self.model.predict(np.array([self.point]))[0]
  File "C:\Users\joelw\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1598, in predict
    data_handler = data_adapter.DataHandler(
  File "C:\Users\joelw\anaconda3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 1100, in __init__
    self._adapter = adapter_cls(
  File "C:\Users\joelw\anaconda3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 263, in __init__
    x, y, sample_weights = _process_tensorlike((x, y, sample_weights))
  File "C:\Users\joelw\anaconda3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.

In [96]:
plt.ion()

In [98]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [148]:
a = encoder.predict(X)

In [149]:
a[1240]

array([0.51041174, 0.        , 2.0535278 , 0.        , 4.0009913 ,
       3.1634362 , 3.9954054 , 6.0741687 , 3.7251441 , 3.414531  ,
       0.        , 0.        , 2.9472818 , 3.9040203 , 0.        ,
       0.        , 4.5115004 , 3.5715692 , 2.9985487 , 3.830607  ,
       3.5655587 , 3.325972  , 4.3161545 , 2.698904  , 2.9865131 ,
       2.8435328 , 3.3138041 , 0.        , 0.        , 4.095569  ,
       0.        , 0.        , 0.        , 0.        , 2.36571   ,
       0.        , 3.402308  , 2.9761605 , 3.241924  , 5.81714   ,
       3.7589602 , 3.6362002 , 0.        , 0.        , 3.6715107 ,
       3.8807273 , 0.        , 0.        , 4.5231676 , 3.6050918 ,
       3.480691  , 3.2945178 , 3.1012812 , 2.9289844 , 3.2878728 ,
       2.5218225 , 3.7343628 , 2.6464753 , 3.5201762 , 0.        ,
       0.        , 3.4910457 , 0.        , 0.        , 0.45053518,
       0.        , 2.0028694 , 0.        , 3.8637674 , 3.0550897 ,
       3.8196774 , 6.1125956 , 3.7323954 , 3.3793654 , 0.     

In [112]:
axs.shape


(2, 4)

In [133]:
encoder.output.shape[1:]

TensorShape([128])

In [134]:
128/4


32.0